# Aula 7 - Modelos de Aprendizado

## Aplicar Redes Bayesianas no Dataset Police

Será feito um classificador bayesiano para prever se uma parada resultará ou não em prisão.

In [121]:
import pandas as pd

In [122]:
csv_path = "../data/police.csv"
df = pd.read_csv(csv_path)

In [123]:
from etl.main import PoliceDatasetEtl

police_dataset_etl = PoliceDatasetEtl(df.copy())
featured_df = police_dataset_etl.clean_transform()
search_type_df = police_dataset_etl.get_search_type_df()

### Modelo Probabilístico - Rede Bayesiana

* Objetivo: Prever se uma parada policial resultará em prisão ou não
* Utilizar classificador `Bernoli Naive Bayes`, como a distribuição do `target`(ser preso ou não) segue uma distribuição binária (`Bernouli`)
* Utilizar a proporção 70% - 30% (70% treino, 30% teste)
* Utilizar validação cruzada `k-fold`

#### Notas:
* **Seleção de features:** Dado a possibilidade de reutilização de trabalhos prévios sob esse dataset, as features utilizadas serão as mesmas descobertas como sendo as que possuem melhor relação de `mutual information score` com o *target* `is_arrested`, conforme utilizado no [notebook da aula 4](./etl_aula_4.ipynb). Portanto:
    * search_type - Foi a única feature deixada como `one-hot-encoding`
    * search_conducted (foi removido uma vez que deixava a acurácia pior)
    * violation_level (foi removido uma vez que não altera a acurácia)
    * driver_race (foi removido uma vez que não altera a acurácia)
* **ETL e Feature Engineering:**
    * Está sendo utilizada a class `PoliceDatasetEtl`, desenvolvida durante as atividades anteriores. Essa classe contém métodos de limpeza, extração e engenharia da feature `violation_level` que quantifica a feature original `violation` em uma variável contínua.
* **Comparação:** Como o modelo de árvore de decisão já foi utilizado no [notebook da aula 4](./etl_aula_4.ipynb), faremos uma comparação do desempenho utilizando o modelo bayesiano para a distribuição de Bernouli.

Selecionando e adequando as features

Algumas features, incluindo o `target` deverão ser discretizados uma vez que são representadas por meio de variáveis categóricas. Para isso utilizaremos a técnica de `one hot encoder`.

In [124]:
# Please, note search type is already transformed with "one-hot-encoding" in search_type_df dataset
selected_features = [
    # "violation_level",
    # "driver_race",
]
target = "is_arrested"

training_df = featured_df[target].copy()

Aplicando `one-hot-encoder`:

In [125]:
# Fazendo o encoding das features categóricas
encoded_training_df = training_df.copy()
category_feature_to_encode = [
    # "driver_race",
]

# Fazendo o encoding da target
y_encoded = y.copy()
y_encoded = y_encoded.astype("category")
y_encoded = y_encoded.cat.codes

for category in category_feature_to_encode:
    encoded_training_df[category] = encoded_training_df[category].astype("category")
    encoded_training_df[f"{category}_encoded"] = encoded_training_df[category].cat.codes

# encoded_training_df.pop("driver_race")

# merge training_df with search_type_df
encoded_training_df = search_type_df.copy()

Fazendo o split do dataset para treino - teste

In [126]:
from sklearn.model_selection import train_test_split

# split training dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    encoded_training_df, y_encoded, test_size=0.3, random_state=42
)

Treinando e medindo a acurácia da `Rede Bayesiana`

In [127]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Inicializando o classificador Naive Bayes
naive_bayes_classifier = BernoulliNB()

# Treinando o classificador Naive Bayes com os dados de treino
naive_bayes_classifier.fit(X_train, y_train)

# Fazendo predições com o classificador Naive Bayes
y_pred_test = naive_bayes_classifier.predict(X_test)

# Medindo a acurácia do classificador Naive Bayes com os dados de teste
acuracia = accuracy_score(y_test, y_pred_test)

print(f"Acurácia: {(acuracia * 100):.2f} %")

Acurácia: 96.87 %


Utilizando validação cruzada: `K-fold`

In [128]:
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.naive_bayes import BernoulliNB

training_df = search_type_df.copy()

# Realizando o encoding do target
y = featured_df.is_arrested
y_encoded = y.copy()
y_encoded = y_encoded.astype("category")
y_encoded = y_encoded.cat.codes

# split training dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(
    encoded_training_df, y_encoded, test_size=0.3, random_state=42
)


# Inicializando o classificador Naive Bayes
naive_bayes_classifier = BernoulliNB()

# Inicializar o KFold para obter os índices de cada fold
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Realizar validação cruzada com 5 folds no conjunto de treinamento
scores = cross_val_score(naive_bayes_classifier, X_train, y_train, cv=kf)

# Imprimir as pontuações de cada fold na validação cruzada
print("Cross-validation scores:", scores)

# Loop sobre os folds para obter os índices dos dados
for fold, (train_index, _) in enumerate(kf.split(X_train)):
    print(f"Fold {fold + 1}:")
    print(f"  - Índices dos exemplos de treinamento: {train_index}")

Cross-validation scores: [0.97119277 0.97127063 0.9687792  0.97033403 0.96854318]
Fold 1:
  - Índices dos exemplos de treinamento: [    0     1     2 ... 64215 64216 64217]
Fold 2:
  - Índices dos exemplos de treinamento: [    2     4     5 ... 64215 64216 64217]
Fold 3:
  - Índices dos exemplos de treinamento: [    0     1     2 ... 64212 64213 64214]
Fold 4:
  - Índices dos exemplos de treinamento: [    0     1     2 ... 64215 64216 64217]
Fold 5:
  - Índices dos exemplos de treinamento: [    0     1     3 ... 64215 64216 64217]


## Conclusões

* Das 4 features que pretendíamos utilizar, somente 1 (processada com `one-hot-encoding`) está sendo usada uma vez que removendo as outras features do modelo não alterava a acurácia, ou alterava positivamente.
* Utilizando o crossvalidation com o `K-fold`, podemos constatar que a acurácia com o modelo treinado em cada fold é similar, ou apresenta baixa variação. Esse fato aumenta a confiança de que o modelo irá ter performance similar em produção uma vez que a distribuição dos dados das features consideradas para o propósito é consistente em cada fold.
* Se comparado com o modelo de árvores de decisão implementado no [notebook da aula 4](./etl_aula_4.ipynb), podemos perceber uma acurácia muito similar, com baixa variação.